In [1]:
import random
import numpy as np
import math
import pandas as pd
width = 20

In [2]:
with open('rowZip.csv', 'r') as file:
    rows = file.readlines()
    file.close()
rows = [[int(st) for st in row.split(',')] for row in rows]
num = len(rows)

In [3]:
with open('colZip.csv', 'r') as file:
    cols = file.readlines()
    file.close()
cols = [[int(st) for st in col.split(',')] for col in cols]
len(cols)

23596

pick = random.sample(range(num), width**2)
random.shuffle(pick)
np.sort(pick)

In [4]:
O = set([])
for i in cols:
    O |= set(i)
len(O)

157

In [5]:
tp = set([])
pick = []
m = width*width
while len(tp) < len(O) and len(pick) < m:
    tmp = 0
    x = None
    for i in range(len(cols)):
        addi = len(set(cols[i]) - tp)
        if addi > tmp:
            tmp = addi
            x = i
    tp |= set(cols[x])
    pick.append(x)

while len(pick) < m:
    tmp = random.randint(0, m-1)
    if tmp not in pick:
        pick.append(tmp)
len(pick)

400

In [6]:
O = set([])
for i in pick:
    O |= set(cols[i])
len(O)

157

def user_ranstr(num):
    userID = ''
    H = 'AB'
    userID += random.choice(H)
    H = '0123456789'
    for i in range(num-1):
        userID += random.choice(H)
    return userID


def item_ranstr(num):
    itemID = ''
    H = 'def'
    itemID += random.choice(H)
    H = '0123456789'
    for i in range(num-1):
        itemID += random.choice(H)
    return itemID

In [7]:
def initial_dataset(n,num,num2):
    #n: grid width, num: length of user/item ID , num2: max_bicluster_width
    bicluster = []
    for i in range(n):
        '''num_user = random.randrange(num2)
        num_item = random.randrange(num2)
        userID = []
        itemID = []
        j = 0
        k = 0
        while j < num_user:
            try:
                user = user_ranstr(num)
                userID.index(user)
            except:
                userID.append(user)
                j= j + 1 
        while k < num_item:
            try:
                item = item_ranstr(num)
                itemID.index(item)
            except:
                itemID.append(item)
                k = k + 1'''
        userID = rows[i]
        itemID = cols[i]
        single_set = [userID,itemID]
        bicluster.append(single_set)
    return bicluster

In [8]:
def reward_func(set1,set2):   
    intersection = 0
    for i in range(len(set2)):
        try:
            set1.index(set2[i])
            intersection = intersection + 1
     #       print(set2[i])
        except:
            pass 
   ## print("交:",intersection)
   # print("並:",(len(set1)+len(set2)-intersection))
    try:
        reward = intersection/(len(set1)+len(set2)-intersection)
    except:
        reward = 1
    return round(reward,3)

In [9]:
def gen_gridworld(width, data):
    left_num = width*width
    tp = 0
    gridworld = [[0 for col in range(width)] for row in range(width)]
    random.shuffle(pick)
    for i in range(width):
        for j in range(width):
            '''num = random.randrange(left_num)
            gridworld[i][j] = data[num]
            data.remove( data[num])
            left_num = left_num - 1'''
            gridworld[i][j] = data[pick[tp]]
            tp += 1
    return gridworld

In [10]:
def gen_reward_grid(gridworld,width):
    grid_reward = [[0 for col in range(width)] for row in range(width)]
    for i in range(width):
        for j in range(width):
            reward=[]
            #print("i=",i,"j=",j)
            if j%width == width-1:
                if i == 0:
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(0)
                elif i == width-1:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(0)                        
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(0)  
                else:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(0)       
            elif j%width == 0:
                if i == 0:
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                        
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))  
                elif i == width-1:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(0)                        
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))  
                else:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                        
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))    
            else: 
                if i == 0:
                    reward.append(0)
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))
                elif i == width-1:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(0)                        
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))
                else:
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                    reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))   
            grid_reward[i][j] = reward
    return grid_reward

In [11]:
def total_reward(grid_reward):
    return np.sum(grid_reward)

In [12]:
def hilclimb(tot_reward,reward_grid,data_grid,width,alpha, beta,iteration):
    i = random.randrange(width)
    j = random.randrange(width)
    i2 = random.randrange(width)
    j2 = random.randrange(width)
    data_grid[i][j], data_grid[i2][j2] =  data_grid[i2][j2], data_grid[i][j]
    reward_update(reward_grid,data_grid,i,j, width)
    reward_update(reward_grid,data_grid,i2,j2, width)
    new_tot_reward = total_reward(reward_grid)
    P_k = 1.0-beta*alpha**iteration
    #print("P_k:",P_k)
    backfire = random.random()
   # print("backfire:",backfire)
    T = 10000000
    flag = (iteration % T == 0)
    if P_k < backfire:
        if flag:
            print("backfire,old:",tot_reward," new:",new_tot_reward)
        return new_tot_reward, 1
    else: 
        if new_tot_reward > tot_reward:
            if flag:
                print("update:",new_tot_reward)
            return new_tot_reward, 1
        else:
            if flag:
                print("non_update:",tot_reward," new:F",new_tot_reward)
            data_grid[i][j],  data_grid[i2][j2] =   data_grid[i2][j2],  data_grid[i][j]
            reward_update(reward_grid,data_grid,i,j, width)
            reward_update(reward_grid,data_grid,i2,j2, width)
            return tot_reward, 0

In [13]:
def reward_update(grid_reward,gridworld,x,y, width):
    x_update = [x-1,x,x+1,x,x]
    y_update = [y,y,y,y-1,y+1]
    for i, j in zip(x_update, y_update):
        reward=[]
        #print("i=",i,"j=",j)
        if i < 0 or i > width-1 or j < 0 or j> width-1:
           # print("i=",i,"j=",j)
            continue
        if j%width == width-1:
            if i == 0:
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(0)
            elif i == width-1:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(0)                        
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(0)  
            else:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(0)       
        elif j%width == 0:
            if i == 0:
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                        
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))  
            elif i == width-1:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(0)                        
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))  
            else:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                        
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))    
        else: 
            if i == 0:
                reward.append(0)
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))
            elif i == width-1:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(0)                        
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))
            else:
                reward.append(reward_func(gridworld[i][j][0],gridworld[i-1][j][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i+1][j][0]))                       
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j-1][0]))
                reward.append(reward_func(gridworld[i][j][0],gridworld[i][j+1][0]))   
        grid_reward[i][j] = reward

In [14]:
width = 20
id_length = 2
max_bicluster_width = 20
beta = 0.4
stop_value = 10000

data = initial_dataset(num,id_length,max_bicluster_width)
data_grid = gen_gridworld(width, data)
reward_grid = gen_reward_grid(data_grid,width)
tot_reward = total_reward(reward_grid)
print(tot_reward)
stop = 0
alpha = 0.99999
iteration = 0

776.1220000000001


In [15]:
stop_value

10000

In [16]:
while True:
    iteration = iteration + 1
    tot_reward, update_bool = hilclimb(tot_reward,reward_grid,data_grid,width,alpha, beta, iteration)
    P_k = 1.0-beta*alpha**iteration
    if update_bool == 1:
        stop = 0
    else:
        stop = stop+1
    if iteration % 10000 == 0:
        print(P_k, tot_reward)
    if 1 - P_k < 1e-7:
        break
    #print("real: ", total_reward(gen_reward_grid(data_grid,width)))
print(tot_reward)

0.6380652137540963 809.942
0.67250802626283 812.1080000000002
0.7036731562204706 826.1500000000001
0.7318725178443317 825.442
0.7573883426483397 831.7920000000001
0.780476004139156 826.4960000000001
0.8013665737056469 840.0880000000001
0.8202691332821546 837.0020000000001
0.8373728679317141 838.0680000000002
0.8528489592927013 855.2440000000001
0.866852298839354 845.7900000000002
0.8795230381032203 845.9520000000001
0.8909879913708328 852.714
0.9013619048964144 850.1500000000001
0.9107486053324516 871.2520000000002
0.9192420388721302 882.894
0.9269272115038236 864.3020000000001
0.9338810397881107 879.2520000000002
0.9401731206722711 876.202
0.9458664280468974 886.4280000000001
0.9510179430160663 887.8500000000001
0.9556792241890882 896.9920000000002
0.9598969237015627 883.676
0.9637132541303047 903.8220000000001
0.967166410975231 917.236
0.970290954936586 906.5400000000001
0.9731181578135083 916.196
0.97567631548584 922.0220000000002
0.9779910311116368 926.7900000000001
0.9800854713747

In [17]:
total_reward(gen_reward_grid(data_grid,width))

1156.3360000000002

In [18]:
data_grid[0][1]

[[90, 106, 227, 274, 306, 308, 332, 349, 353, 358, 398, 438, 468, 648],
 [0,
  2,
  3,
  4,
  6,
  9,
  10,
  12,
  15,
  16,
  18,
  20,
  21,
  22,
  36,
  48,
  50,
  57,
  67,
  74,
  77,
  79,
  101,
  111]]

In [19]:
import pickle

f1 = open("dic", "rb")
f2 = open("dic_inv", "rb")
dic = pickle.load(f1)
dic_inv = pickle.load(f2)
f1.close()
f2.close()

In [20]:
real_grid = [[[data_grid[k][j][0], [dic_inv[i] for i in data_grid[k][j][1]]] for j in range(width)] for k in range(width)]

In [21]:
f = open("grid2", "wb")
pickle.dump(real_grid, f)
f.close()

In [22]:
f = open("grid2", "rb")
pickle.load(f) == real_grid
f.close()

In [23]:
real_grid

[[[[90, 106, 274, 306, 332, 349, 353, 359, 398, 438, 468, 648, 654, 707],
   [318,
    296,
    593,
    2571,
    260,
    589,
    2959,
    1196,
    1210,
    50,
    4993,
    1198,
    2858,
    7153,
    1270,
    5952,
    1265,
    1291,
    6539,
    7361,
    6874,
    3793,
    7438,
    5349]],
  [[90, 106, 227, 274, 306, 308, 332, 349, 353, 358, 398, 438, 468, 648],
   [318,
    296,
    593,
    2571,
    527,
    2959,
    1196,
    1210,
    4993,
    1198,
    2858,
    7153,
    1270,
    5952,
    2762,
    1265,
    1291,
    6539,
    7361,
    4886,
    6377,
    3793,
    4995,
    5989]],
  [[27, 90, 106, 227, 274, 308, 332, 349, 353, 358, 398, 438, 468, 648],
   [318,
    356,
    296,
    593,
    527,
    1196,
    1210,
    4993,
    1198,
    2858,
    7153,
    1270,
    5952,
    2762,
    1291,
    1580,
    6539,
    7361,
    4886,
    6377,
    3793,
    4995,
    5989,
    5816]],
  [[27, 82, 90, 106, 227, 274, 308, 332, 349, 353, 398, 464, 587, 648